In [ ]:
import h5py
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
def getdatasetname(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split("/")[-1]
    temp = filename_without_dir.split("_")[:-1]
    datasetname = "_".join(temp)
    return datasetname

In [ ]:
def get_label(filename):
    if "rest" in filename:
        label = 0
    elif "math" in filename:
        label = 1
    elif "memory" in filename:
        label = 2
    elif "motor" in filename:
        label = 3
    return label

In [ ]:
def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)

    return dataset, labels

In [ ]:
def scale(matrix):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(matrix)
    scaled_data = scaler.transform(matrix)
    return scaled_data

In [ ]:
def donwsample(dataset, frequency):
    downsampled_dataset = []

    for i in range(0, dataset.shape[1], 2034):
        second = dataset[:, i : i + 2034]
        subsample = []

        for j in range(0, 2034, int(2034 / frequency)):
            if j < second.shape[1]:
                measurement = second[:, j]
                subsample.append(measurement)

        downsampled_dataset.extend(subsample)

    return np.array(downsampled_dataset).T

In [ ]:
filename_path = "./data/Intra/train/rest_105923_1.h5"
with h5py.File(filename_path, "r") as f:
    dataset_name = getdatasetname(filename_path)
    matrix = f.get(dataset_name)[()]
    print(type(matrix))
    print(matrix.shape)

In [ ]:
dataset = get_all_matrices(dir_path)[0]
# subjects x sensors x time
dataset_X = np.stack(dataset)
labels = get_all_matrices(dir_path)[1]

### Data Pipeline

1. Get dataset
2. Downsample data
3. Scale Data
4. Train Model


Model setup
